# Homework 1

## Exersice 03
Exercise 3 (Virus test) A medical company is assessing a newly developed
test for a certain virus. The so called false negative rate is small: if you have
contracted the virus, the probability that the test returns a positive result is
0.999. The false positive rate is also small: if you do not have the virus, the
probability that the test returns a positive result is only 0.005. Assume that 2%
of the population has contracted the virus. If a person chosen uniformly from
the population is tested and the result comes back positive, what is the probability
that the person has the virus?

In python: Draw N = 10^4 pseudo random samples representing the infection
status of N individuals. Use these samples and the statistics described above for
the virus test to draw pseudo random samples representing a positive or negative
virus test for each individual and verify your results from above.
### Answers:

\begin{equation}
 \frac{0.999 \cdot 0.02}{0.999 \cdot 0.02+ 0.005\cdot (1-0.02)} = 0.80
\end{equation}


### Solution:
we can describe the scenario using bayes theroem by the following formulas
\begin{equation}
P(x|y) = \frac{P(y|x) \cdot P(x)}{P(y)} = \frac{0.999 \cdot 0.02}{P(y)}
\end{equation}

\begin{equation}
 \frac{0.999 \cdot 0.02}{0.999 \cdot 0.02+ 0.005\cdot (1-0.02)} = 0.80
\end{equation}


In [65]:
import numpy as np
import random

# Probabilities
pinf = 0.02  # Probability of being infected
fnr = 0.001  # False negative rate
fpr = 0.005  # False positive rate
Samples = 10**4  # Number of samples

# Generate random samples
status = [1 if random.random() < pinf else 0 for _ in range(Samples)]

# Simulation
test_results = []
for s in status:
    if s == 1:
        # If  person is infected
        test_result = 1 if random.random() < (1 - fnr) else 0
    else:
        # If person not infected
        test_result = 1 if random.random() < fpr else 0
    test_results.append(test_result)



pt = sum(test_results)
ptwv = sum([1 for s, t in zip(status, test_results) if s == 1 and t == 1])

Phv = ptwv / pt

print("Probability that a person has the virus given a positive test result:", Phv)


0.8230088495575221


## Exercise 8 (Group Testing)
 A blood test is to be performed on n individuals. Each person can be tested separately, but this is expensive. An alternative
strategy is to pool and analyze together the samples of k people. If the test
is negative, this one test suffices for the group of k people. If the test is positive,
then each of the k persons must be tested individually, which results in k + 1
total tests for the k people. Suppose we create n/k disjoint groups of k people
(where k divides n) and use the pooling method. Assume that each person has
positive result on the test independently with probability p.
• What is the probability that the test for a pooled sample of k people will
be positive? What is the expected number of tests necessary? python:
Confirm your calculations by drawing n = 102 pseudo random test results
and dividing the test results into groups of k = 10 with p ∈ {0.02, 0.2, 0.6}.
Average your results over 104 iterations.
• Describe how to find the best value of k. Derive an approximation as-
suming p is close to 0. python: Find the optimal integer k for p ∈
{0.02, 0.2, 0.6} by assuming k < 100. Compare with your approximation.
20
Giuseppe Durisi, Probability and Statistical Learning using Python August 23, 2023
• For which values of p is pooling better than just testing every individual?
Again, derive an approximation assuming p is small and plug in your
approximation of the best value of k. python: Using your results from the
previous parts, test if pooling is better or not for n = 102, 0.01 ≤ p ≤ 0.5
and the best integer value of k (assume k < 100). Compare with your
approximation. You might want to plot some function for different values
of p


### Answers: 
Fill in your answers here so that Carl can do a quick check if they are right/wrong

* ...


### Solution:



In [2]:
# Placeholder for python part of exersice ZZ
